In [ ]:
%run ralstats.py

# RAL Hands-on "Statistical Analysis Lab" Session

In this session you will learn how to perform the most common statistical analysis tasks, building things up from two fundamental operations: *fitting* and *generating*.

### Learning Objectives

In this session we will go cover the operations involved in the calculation of:

    1. Exclusion Limits
    2. Discovery p-values
    3. Parameter estimation, asymmetric errors, and impact calculation

In your real work it is likely that you will utilise one of the many "statistical toolkits" that have been developed that will automate the calculation and visualization of these analyses for you. The objective of this session is to give you the **confidence that you understand what the toolkits *should* be doing** under the hood, and know how to interrogate the calculations they are making by **asking sensible questions** of those toolkits. This is an important skill because there are lots of ways in which these analyses can go wrong, and so **understanding what the calculations are doing is essential for enabling you to effectively debug problems.**

The tutorial uses the RooFit statistical modelling toolkit, but minimal experience of the classes of this toolkit are required.

### Basic functions

The following functions are provided for you in this session:

| function | arguments | description |
|----------|-----------|---------------|
| `getWorkspace(day,month)` | `day`: Your birthday day of month (1-31)<br>`month`: Your birthday month of year (1-12) | Returns a `RooWorkspace` containing the model and data you will use |
| `plot(model,dataset,globs)` | `model`: the model (a `RooAbsPdf`)<br>`dataset`: the dataset (a `RooDataSet`)<br>`globs`: the global observable values of the dataset (a `RooArgSet`) | A visualization of the model and dataset, taking into account the current values of the parameters of the model. It shows one plot for each channel of the model, and a plot for each global observable overlaying its corresponding probability distribution in the model. |
| `fit(model,dataset,globs)` | As above | Returns a `RooFitResult` from fitting the model to the data. The floating parameters of the model are the ones that are not marked constant at the time the function is called.
| `generate(model, fitResult=None, expected=False)` | `model`: As above<br>`fitResult`: The `RooFitResult` used to set all the parameters of the model to before generating<br>`expected`: if `True` will return the asimov dataset rather than a random toy | Returns a `RooDataSet` and a `RooArgSet` of the dataset and globs. Note: if no fit result is passed, the current values of the parameters are used. |

There are also some functions for providing asymptotic p-values, which we will encounter later in the tutorial ...


## Getting to know our model

There unfortunately isn't time in this session to teach you about building models. So we provide you one instead. Access it with the `getWorkspace` function and lets familiarize ourself with it:

In [ ]:
w = ...

<span style="color:orange">**Task: Determine the name of the top-level pdf in the workspace, and the name of the dataset in the workspace (there is only 1) ...**</span>

In [ ]:
pdfName = None
dataName = None

...

<span style="color:orange">**Task: Access the model, dataset, and global observables snapshot (name is the same as the dataset name) from the workspace ...**</span>

In [ ]:
model = ...
obsData = ...
obsGlobs = ...

In [ ]:
plot(model,obsData,obsGlobs)

Discussion: What are we seeing?

<span style="color:orange">**Task: Access the variables of the model and understand what they represent. Which are observables, global observables, and parameters? Change the parameter values and replot the model to understand their effect. Generate some toy datasets and visualize them too, and try running a fit...**</span>

In [ ]:
...


<span style="color:orange">**Task: Assign the variables representing signal mass and signal strength to the python variables below...**</span>

In [ ]:
mass_var = ...
mu_var = ...

## Hypothesis Testing: Exclusions and Discovery

Exclusions and Discovery plots are just a way of presenting the results of a collection of Hypothesis Tests. A Hypothesis Test is really the process of calculating a p-value and seeing whether its less than or greater than a  critical value (0.05 in the case of 95% CL). The procedure for making these plots is as follows:

  1. Define a `Hypothesis Space`, which involves deciding which parameters of the BSM model we want to scan over.
      (elsewhere people call this a *parameter grid*):
  2. Choose a `Test Statistic` to perform hypothesis tests with, the normal choice is:
      * Exclusions: A one-sided *capped-above* Profile Likelihood Ratio Test Statistic ($q_\mu$ or $\tilde{q}_\mu$)
      * Discovery: A one-sided *capped-below* Profile Likelihood Ratio Test Statistic $q_0$
  3. Choose a definition of `P Value` to use, there are three types:
      * null p-value: The p-value under the null hypothesis (the hypothesis being tested)
         * In exclusion tests the null hypothsesis is a particular s+b hypothesis, and null p-value is sometimes then called the CLs+b p-value.
         * In discovery tests the null hypothesis is the bkg-only hypothesis, and the null p-value is sometimes called the $p_0$ p-value.
      * alt p-value: The p-value under an alternative hypothesis (only relevant for exclusions), also calld the CLb p-value
      * CLs p-value: The ratio of the above two p-values
  4. Find the contour of `Hypothesis Points` in your `Hypothesis Space` where your chosen p-value equals the critical value (0.05 for a 95% confidence limit). This is usually done by selecting a grid of points in the space and computing the p-value for each (aka doing a hypothesis test), then interpolating between them to give you a contour. 
  5. `Expected` contours can also be computed by throwing toys from the expected hypothesis and computing the median p-value of these toys at each hypothesis point. The contour is drawn as before by interpolating these median p-values across the space. 1 sigma and 2 sigma "bands" can be computed by examining a different p-value than the median one, as we will discuss later.
      * Note that the expected contour is often instead estimated by using the *Asimov Dataset* which corresponds to the expected data under the alternative hypothesis. This is not quite the same as using toys but usually gives a very similar result.

### 1. The Hypothesis Space

Our hypothesis space will span signal mass in the x-axis, and signal strength in the y-axis. We will leave signal width fixed. Note that you don't have to choose signal strength as one of your space axis parameters, SUSY analyses for example often scan over a pair of masses and leave signal strength fixed to 1. 

We'll use a `TGraph` to keep track of which points we are running hypothesis tests for

In [ ]:
hypoSpace = ROOT.TGraph()
hypoSpace.SetTitle("Hypothesis Space;M [GeV];\mu")

# start with just some points @ mu=1
mass_var.setVal(90.0)
mu_var.setVal(1.0)

while mass_var.getVal()<=150.0:
    hypoSpace.AddPoint(mass_var.getVal(),mu_var.getVal())
    mass_var.setVal(mass_var.getVal()+5.0)

myPlot = ROOT.gROOT.MakeDefCanvas()
hypoSpace.Draw("AP")
myPlot.Draw()

### 2. The Test Statistic 

The test statistics we are using today are all based on the *(two-sided) profile likelihood ratio*, $t_\mu$:

$$
t_\mu = -2\text{ln}\left( \frac{L(\mu,\hat{\hat{\theta}})}{L(\hat{\mu},\hat{\theta})}\right)
$$

Here's a table of different test statistics based on the profile likelihood ratio (the notation comes from the the [Asymptotic formulae paper](https://arxiv.org/pdf/1007.1727.pdf)):

||$\mu$ unbounded|$\mu$ lower-bounded @ 0|
| --: | :-: | :-: |
|*one-sided capped-above profile likelihood ratio*|$$
q_\mu = \begin{cases}
    t_\mu \text{ if $\hat\mu < \mu$,} \\
    0 \text{ if $\hat\mu \geq \mu$}.
    \end{cases}
$$|$$
\tilde{q}_\mu = \begin{cases}
    -2\text{ln}\left( \frac{L(\mu,\hat{\hat{\theta}})}{L(0,\hat{\hat{\theta}}(0))}\right) \text{ if $\hat\mu < 0$,} \\
    t_\mu \text{ if  $0 \leq \hat\mu < \mu$,} \\
    0 \text{ if $\hat\mu \geq \mu$}.
    \end{cases}
$$|
|*one-sided capped-below profile likelihood ratio*|$$
q_0 = \begin{cases}
    t_{\mu=0} \text{ if $\hat\mu > 0$,} \\
    0 \text{ if $\hat\mu \leq 0$}.
    \end{cases}
$$|-|

    
Question: Which test statistic do are we going to use for our exclusion limits?

Question: Are the one-sided test statistics continuous or do they have a jump in them? What happens when $\hat{\mu} = \mu$ ? 

<span style="color:orange">**Task: Implement a function that can calculate these profile likelihood test statistics (for our particular case of bounding on the parameter of interest) ...**</span>

In [ ]:
def pll(mu_test, data, globs, oneSided=False, cappedAbove=True):
    # use mu_var variable you set above
    ....
    

Now we use your function to evaluate the test statistic with the observed data at $\mu=1$ at different values of signal mass.

In [ ]:
c = ROOT.TCanvas()
c.DivideSquare(4)

obs_plls = ROOT.TGraph()
obs_plls.SetTitle("One-sided Profile Likelihood Test Statistic;M [GeV];\\tilde{q}_{\mu=1}")

for i in range(hypoSpace.GetN()):
    mass_var.setVal(hypoSpace.GetPointX(i))
    obs_pll = pll( ... )
    obs_plls.AddPoint(mass_var.getVal(),obs_pll)
    
c.SetSelectedPad( c.GetPad(1).cd() )
obs_plls.DrawClone("ALP")
c.Draw()

Questions:
   * What is the meaning of this test statistic value? What does a small or big value mean?
   * Why this test statistic for hypothesis testing?

### 3. The P-Value

Now, this is the computationally difficult bit. We need to know the p-value of our observed test statistic values. For the CLs+b p-value this is the fraction of toys generated under the null hypothesis that had a test statistic value greater than the one we observed. The CLb is similar but with toys generated under the alternative hypothesis. 

<span style="color:orange">**Task: Implement a function that can generate the test statistic distribution from toys for a single hypothesis point and use it to estimate the p-value for a single point ...**</span>

In [ ]:
def pvalue(nToys, mu_test, mu_true, obs_ts):
    hist = ROOT.TH1D("toys","",100,0,2*obs_ts)
    hist.SetDirectory(0)
    
    # to generate toys under the "true" hypothesis we need the fitResult of the obs data to that specific hypothesis
    ...
    true_fit = fit(model,obsData,obsGlobs)
        
    nUp = 0
    for i in tqdm.tqdm(range(nToys)):
        toy_ts = ...
        nUp += (toy_ts >= obs_ts) # could this fail?
        hist.Fill(toy_ts)
    
    # convert hist into a PDF by normalizing and dividing by bin width
    hist.Scale(1./(nToys*hist.GetBinWidth(1)))
    
    # leave mu at its original value
    mu_var.setVal(mu_test)
    
    hist.SetTitle("M = {} GeV, \mu' = {} : pval = {};q_{{\mu={}}}".format(mass_var.getVal(),mu_true,nUp/nToys,mu_test))
    
    return nUp/nToys , hist 

# Here we will use your function to generate the test statistic distribution for the 5th point in the hypospace
mu_var.setVal(hypoSpace.GetPointY(5))
mass_var.setVal(hypoSpace.GetPointX(5))
pval_clsb, nullHist = pvalue(20,mu_var.getVal(),mu_var.getVal(),obs_plls.GetPointY(5))
pval_clb, altHist = pvalue(20,mu_var.getVal(),0,obs_plls.GetPointY(5))

nullHist.SetLineColor(ROOT.kBlue);altHist.SetLineColor(ROOT.kRed)
myPlot.cd(); myPlot.SetLogy()
nullHist.DrawClone("histe");altHist.DrawClone("histe same")
myPlot.BuildLegend()
l = ROOT.TLine();l.SetLineWidth(2);l.DrawLine(obs_plls.GetPointY(5),0,obs_plls.GetPointY(5),1) # obs ts value
myPlot.Draw()

Questions:

  * Why do the alt hypothesis toys take longer to run than the null hypothesis toys?
  * How would we extract the expected p-value from this plot?
  * Should p-values have an uncertainty?
  * How many toys?

### A shortcut to p-values: Asymptotic formulae



In [ ]:
help(asymptotic_pvalue_qmu)

<span style="color:orange">**Task: Write a function to fill a graph with the PDF of the asymptotic p-value. Use it to explore the shape of the function for different values of $\sigma_{\hat{\mu}}$ and $\mu$ ...**</span>

In [ ]:
def asymptotic_graph(mu_test, mu_prime, sigma_mu):
    out = ROOT.TGraph(); out.SetLineWidth(2); out.SetTitle(";q_{{\mu={}}}".format(mu_test))
    
    ... code using asymptotic_pvalue_qmu to fill points in a TGraph ...
    
    return out

# here we will draw the asymptotic distribution for mu_test=4 but different choices of mu_true (left:0 and right:4) and
# different choices of sigma_mu (0.5 = black,1 = red,2 = blue)
myPlot.cd()
myPlot.Clear()
myPlot.Divide(2,1)
myPlot.cd(1)
asymptotic_graph(4,0,0.5).DrawClone("AL")
asymptotic_graph(4,0,1).DrawClone("L").SetLineColor(ROOT.kRed)
asymptotic_graph(4,0,2).DrawClone("L").SetLineColor(ROOT.kBlue)
myPlot.cd(2).SetLogy()
asymptotic_graph(4,4,0.5).DrawClone("AL")
asymptotic_graph(4,4,1).DrawClone("L").SetLineColor(ROOT.kRed)
asymptotic_graph(4,4,2).DrawClone("L").SetLineColor(ROOT.kBlue)
myPlot.Draw()

Questions:
  * How does the asymptotic distribution depend on $\sigma_{\hat{\mu}}$ parameter?
  * What happens when plotting the null hypothesis distribution for $\mu=0$? Why? 

To use the asymptotic formulae we will need to estimate $\sigma_{\hat{\mu}}$, which we can do with the asimov dataset (the expected dataset) and using the formula:
$$
\sigma_{\hat{\mu}}\approx\frac{|\mu - \mu'|}{\sqrt{t_\mu( D_{exp}(\mu') )}}
$$

where $\mu$ is the hypothesis being tested, $\mu'$ is the 'true' hypothesis, $t_\mu(...)$ is the regular (two-sided) profile likelihood ratio, and $D_{exp}(\mu')$ is the asimov dataset under the $\mu'$ hypothesis.

<span style="color:orange">**Task: Write code to estimate $\sigma_{\hat{\mu}}$ for the hypothesis points in our hypothesis space:**</span>

In [ ]:
sigma_mus = ROOT.TGraph()
sigma_mus.SetTitle("std deviation of \hat{\mu};M [GeV];\sigma_{\mu}")

# Generate the asimov (expected) dataset for the mu=0 hypothesis
# need to define the appropriate 'fit result' representing that hypothesis....
...
cfit = ...
asimovData = generate(model,cfit,True)

import math
for i in range(hypoSpace.GetN()):
    mass_var.setVal(hypoSpace.GetPointX(i))
    mu_test = hypoSpace.GetPointY(i)
    asimov_pll = pll(...)
    sigma_mu = ...
    sigma_mus.AddPoint(hypoSpace.GetPointX(i),sigma_mu)
    
c.cd(2)
sigma_mus.Draw("ALP")
c.Draw()

Discussion: Interpretation of $\sigma_{\hat{\mu}}$ distribution...
Question: Cause of uptick in the $\sigma_{\hat{\mu}}$ at the end of the mass observable space?

Now that we have computed $\sigma_{\hat{\mu}}$ values, plot the asymptotic distribution on top of the toy distribution you generated earlier for a single hypothesis point to assess if for that particular point if we are in the "asymptotic regime" ...

In [ ]:
nullGr = asymptotic_graph(1,1,sigma_mus.GetPointY(5))
altGr = asymptotic_graph(1,0,sigma_mus.GetPointY(5))
nullGr.SetLineColor(nullHist.GetLineColor())
altGr.SetLineColor(altHist.GetLineColor())

myPlot.cd()
nullHist.DrawClone("histe");altHist.DrawClone("histe same")
nullGr.Draw("L"); altGr.Draw("L")
myPlot.Draw()

<span style="color:orange">**Task: Now use the asymptotic formulae function (`asymptotic_pvalue_qmu`) to graph the asymptotic CLs p-value across all the hypothesis points .. hint: you need to use your obs_plls and sigma_mus:**</span>

In [ ]:
obs_pvals = ROOT.TGraph()
obs_pvals.SetTitle("Observed CLs p-value;M [GeV];Observed p_{CLs}")

for i in range(hypoSpace.GetN()):
    psb = asymptotic_pvalue_qmu(...)
    pb = asymptotic_pvalue_qmu(...)
    obs_pvals.AddPoint(hypoSpace.GetPointX(i),psb/pb)
    
c.cd(3)
obs_pvals.Draw("ALP")
ROOT.gPad.SetGridy()
c.Draw()

## 4. Finding the P-Value Contours

Now that we have a quick way to estimate p-values, our objective is to find the contour of hypothesis points in a full hypothesis where the p-value equals the critical value. 

Question: For a 95% CL limit, what is the critical value?

<span style="color:orange">**Task: Define a full 2D hypothesis space, scanning mu values from 0.1 to 4.9 in steps of 0.2, and mass values from 90-150 in steps of 5 GeV:**</span>

In [ ]:
fullHypoSpace = ROOT.TGraph()
fullHypoSpace.SetTitle("Full Hypothesis Space;M [GeV];\mu")
... need a loop within a loop ...

<span style="color:orange">**Task: Use the asymptotic formulae function to compute the p-values across the full hypothesis space:**</span>

In [ ]:
# now get p-value at every point in hypospace and plot contour where equals 0.05
full_obs_pvals = ROOT.TGraph2D();full_obs_pvals.SetTitle("Observed CLs p-values;M [GeV];\mu")
full_sigma_mu = ROOT.TGraph2D(); # use this to save all the sigma_mu values

for i in tqdm.tqdm(range(fullHypoSpace.GetN())):
    mass_var.setVal(fullHypoSpace.GetPointX(i));
    asimov_pll = ...
    sigma_mu = ...
    full_sigma_mu.AddPoint(fullHypoSpace.GetPointX(i),fullHypoSpace.GetPointY(i),sigma_mu)
    obs_pll = ...
    
    p_null = asymptotic_pvalue_qmu(...)
    p_alt = asymptotic_pvalue_qmu(...)    
    p_cls = p_null/p_alt if p_alt>0 else (1 if p_null==p_alt else float('nan')) # a bit of protection against p_null=p_alt=0
    full_obs_pvals.AddPoint(fullHypoSpace.GetPointX(i),fullHypoSpace.GetPointY(i),p_cls)

In [ ]:
# visualize our p-value graph
myPlot.cd()
full_obs_pvals.Draw("LEGO");
myPlot.Draw()

These p-value 2D graphs are used to extract the 95% CLs limits. This is done by finding the contours where the p-value equals 0.05.

Question: What piece of information would we need in order to compute an expected limit contour?

<span style="color:orange">**Task: Use the `asymptotic_expected_qmu` function provided in this lab to compute the p-value graphs for $-2\sigma$ to $2\sigma$:**</span>

In [ ]:
help(asymptotic_expected_qmu)

In [ ]:
# we will keep the collection of expected p-values in a dictionary of graphs
# a defaultdict is just a way to have a dictionary that will automatically create a new TGraph2D for us
# when we access a new key in the dictionary
from collections import defaultdict
full_exp_pvals = defaultdict(ROOT.TGraph2D)

for i in tqdm.tqdm(range(fullHypoSpace.GetN())):
    mass_var.setVal(fullHypoSpace.GetPointX(i));
    sigma_mu = full_sigma_mu.GetZ()[i] 
    
    def doExpected(nSigma):
        exp_p_true = ROOT.Math.normal_cdf(nSigma) # the p-value of the nSigma toy under the true hypothesis
        exp_pll = ...
        p_null = ...
        p_alt = ...
        full_exp_pvals[nSigma].AddPoint(fullHypoSpace.GetPointX(i),fullHypoSpace.GetPointY(i),p_null/p_alt if p_alt>0 else (1 if p_null==p_alt else float('nan')))
        if nSigma > 0: doExpected(-nSigma)
    
    doExpected(0); doExpected(1); doExpected(2)

The following code block will do the contour finding from your p-value graphs and visualize the results for you.

Discussion point: Interpolating significances rather than p-values 

In [ ]:
# This code is one way to estimate the pvalue=0.05 contours in your graphs. 
# It will also combine the +/-1sigma and +/-2sigma contours so that they can be visualized as 
# green and yellow 'bands' respectively

c.cd(4);
obs_limit = full_obs_pvals.GetContourList(0.05).At(0)
exp_limit = full_exp_pvals[0].GetContourList(0.05).At(0)
exp2_limit = full_exp_pvals[2].GetContourList(0.05).At(0)
expm2_limit = full_exp_pvals[-2].GetContourList(0.05).At(0)

# combine the two contours so can draw as a 'band'
exp2_limit.Sort();expm2_limit.Sort(ROOT.TGraph.CompareX,False)
l = ROOT.TList(); l.Add(expm2_limit)
exp2_limit.Merge(l)

exp1_limit = full_exp_pvals[1].GetContourList(0.05).At(0)
expm1_limit = full_exp_pvals[-1].GetContourList(0.05).At(0)

# likewise for 1sigma 'band'
exp1_limit.Sort();expm1_limit.Sort(ROOT.TGraph.CompareX,False)
l = ROOT.TList(); l.Add(expm1_limit)
exp1_limit.Merge(l)

exp2_limit.SetFillColor(ROOT.kYellow)
exp1_limit.SetFillColor(ROOT.kGreen)
exp_limit.SetLineStyle(2)
exp2_limit.SetTitle("95\% CLs Limit;M [GeV];\mu");
exp2_limit.SetMinimum(0.0)
exp2_limit.Draw("AF")
exp1_limit.Draw("F")
exp_limit.Draw("L")
obs_limit.Draw("L")

c.Draw()

Discussion: It still took a while to get the p-value of every point in the hypothesis space. How could we be even more efficient with our exclusion limit calculation? 

## Discovery Hypothesis Testing

This is a test of the SM (bkg only) hypothesis, using the "one sided capped-below" profile likelihood ratio as a test statistic, $q_0$, and the null hypothesis p-value. 

<span style="color:orange">**Task: Create an appropriate discovery hypothesis space:**</span> 

In [ ]:
dHypoSpace = ROOT.TGraph()
dHypoSpace.SetTitle("Discovery Hypothesis Space;M [GeV];\mu")
mu_var.setVal( ... )
mass_var.setVal( ... )
while mass_var.getVal()<= ... :
    ...

The asymptotic formula for the q0 test statistic is coded in the function `asymptotic_pvalue_q0` and there is also the function `asymptotic_expected_q0` for the value of $q_0$ corresponding to a particular p-value.

In [ ]:
help(asymptotic_pvalue_q0)
help(asymptotic_expected_q0)

Note that in discovery tests it appears that the asymptotic p-value distribution doesn't depend on $\sigma_\mu$, so we can just use any value.

<span style="color:orange">**Task: Compute the null hypothesis p-values and the expected null hypothesis p-values assuming a true hypothesis of $\mu=1$:**</span> 

In [ ]:
null_pvals = ROOT.TGraph()
null_pvals.SetTitle("Null Hypothesis P-Values;M [GeV];pval")
from collections import defaultdict
exp_pvals = defaultdict(ROOT.TGraph)

for i in tqdm.tqdm(range(dHypoSpace.GetN())):
    mass_var.setVal(dHypoSpace.GetPointX(i));
    mu_var.setVal(1); mu_var.setConstant(True)
    # what is mu_test? what is mu_true?
    sigma_mu = ...
    
    obs_pll = ...
    
    p_null = ...
    
    null_pvals.AddPoint(dHypoSpace.GetPointX(i),p_null)
    
    def doExpected(nSigma):
        nSigmaPValue = ...
        
        exp_pvals[nSigma].AddPoint(dHypoSpace.GetPointX(i),...)
        if nSigma > 0: doExpected(-nSigma)
        
    doExpected(0);doExpected(1)

# The rest of this code is just plotting the observed p-values (as a line) and the expected p-values (as a blue band)
myPlot.cd().SetLogy()

exp_pvals[-1].Sort(ROOT.TGraph.CompareX,False)
l = ROOT.TList(); l.Add(exp_pvals[-1])
exp_pvals[1].Merge(l)

exp_pvals[1].SetFillColor(ROOT.kCyan)
exp_pvals[1].SetTitle("Null Hypothesis p-values;M [GeV];pval")
exp_pvals[1].Draw("AF")

exp_pvals[0].SetLineStyle(2)
exp_pvals[0].Draw("L")
null_pvals.Draw("LP")
l = ROOT.TLine(); l.SetLineStyle(2); l.SetLineColor(ROOT.kRed)
def drawSigmaLine(nSigma):
    l.DrawLine(dHypoSpace.GetPointX(0),ROOT.Math.normal_cdf_c(nSigma),dHypoSpace.GetPointX(dHypoSpace.GetN()-1),ROOT.Math.normal_cdf_c(nSigma))
for i in range(1,6): drawSigmaLine(i)
myPlot.Draw()

## Measurement

Now that you may have discovered something we proceed to measurements. To get a measurement of a parameter with an assigned error we need to scan the profile likelihood ratio...

<span style="color:orange">**Task: Scan the PLR of the mass parameter...**</span> 

In [ ]:
pll_mass = ROOT.TGraph()
pll_mass.SetTitle(";M [GeV];-2ln\Lambda")

mu_var.setConstant(False) # question: difference between floating mu vs keeping it constant
mu_var.setVal(1)
mass_var.setConstant(False)
mass_var.setVal(135) # sensible starting point
ufit = fit(model,obsData,obsGlobs)

mass_var.setConstant(True); mass_var.setVal(90)
while mass_var.getVal()<=150:
    ... need to evaluate the 'conditional fit' to data where mass is held constant ... 
    pll = ...
    if pll < 3: # will only plot the PLR points that are less than 3
        pll_mass.AddPoint(mass_var.getVal(),pll)
    mass_var.setVal(mass_var.getVal()+(0.2 if pll<3 else 1))

myPlot.cd()
ROOT.gPad.SetLogy(False)
pll_mass.Draw("ALP")
ROOT.gPad.Draw()

<span style="color:orange">**Task: Compare and contrast the error from the hessian (in the unconditional fit) to the *minos error*...**</span> 

In [ ]:

down = None
up = None
x = pll_mass.GetPointX(0) # start at left-hand edge of pll_mass graph

# scan across the mass finding where the graph becomes equal to 1 ...
# Note: can use graph.Eval(x) to interpolate between points

...
        
ufit.Print() # will show the hessian error

up -= ufit.floatParsFinal().find("sig_mass").getVal()
down -= ufit.floatParsFinal().find("sig_mass").getVal()

print("up=",up,"down=",down)

Finally, lets compute the "impact" of one parameter on another.

<span style="color:orange">**Task: Complete the function below that computes the impact of one parameter with another...**</span> 

In [ ]:
# just do the "post-fit" impact here
def impact(ufit,poiName,npName):
    np_nom = ufit.floatParsFinal().find(npName).getVal()
    np_err = ufit.floatParsFinal().find(npName).getError()
    
    np = model.getVariables()[npName]
    
    # ensure all parameters of the ufit are floating
    # select the parameters from the model that are in the floatParsFinal list of ufit ...
    # (hint: use getVariables(), selectCommon() )
    floatPars = ...
    floatPars.setAttribAll("Constant",False) # makes them all floating
    
    # run fit where NP is moved to nom+err value and held const, and likewise at nom-err
    
    ...
    upfit = ...
    ...
    downfit = ...
    
    
    poi_nom = ufit.floatParsFinal().find(poiName).getVal()
    return (upfit.floatParsFinal().find(poiName).getVal() - poi_nom) , (downfit.floatParsFinal().find(poiName).getVal() - poi_nom)

print("sig_mass:",impact(ufit,"mu","sig_mass"))
print("alpha_par:",impact(ufit,"mu","alpha_par"))

Question: How is impact related to covariance?

In [ ]:
ufit.covarianceMatrix().Print()

In [ ]:
ufit.Print()

### What was the real mass of the particle?

Run `whatIsTheAnswer(day,month)` to find out what was the actual BSM signal mass in your case ... 

In [ ]:
whatIsTheAnswer(...,...)